In [1]:
# We are importing the required libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Checking versions
print ("--Versions--")

print ("Numpy: ", np.__version__)
print ("Pandas: ", pd.__version__)
print ("matplotlib: ", matplotlib.__version__)
print ("Tensorflow: ", tf.__version__)

--Versions--
Numpy:  1.21.6
Pandas:  1.3.5
matplotlib:  3.5.3
Tensorflow:  1.15.5


In [3]:
# Input data read from tensorflow library
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Initializing Weights

In [4]:
# This function will return a Variable of weights created randomly with normal distribution
def initialize_weights(filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape, stddev=0.1)
    return (tf.Variable(init_random_dist))


## Initializing Bias

In [5]:
# This function will return a Variable of bias with constant values
def initialize_bias (bias_shape):
    initial_bias_vals = tf.constant(0.1, shape= bias_shape)
    return (tf.Variable(initial_bias_vals))

## Set up Convolutional Layer and Perform Convolution Computation: Dot Product (x * W)


In [6]:
# function to create Convolution layer to perform convolution using dot product
def create_convolution_layer_and_compute_dot_product(inputs, filter_shape):
    filter_initialized_with_weights = initialize_weights(filter_shape)
    conv_layer_outputs = tf.nn.conv2d(inputs, filter_initialized_with_weights, strides=[1,1,1,1], padding='SAME')
    return (conv_layer_outputs)
    

## Set up a ReLU Layer and Perform Computation: Dot Product + Bias (x.W +b)


In [7]:
# function to create a ReLU layer to perform convolution which means computes dot product on input and weights then sums bias
def create_relu_layer_and_compute_dotproduct_plus_b (inputs, filter_shape):
    b = initialize_bias([filter_shape[3]])
    relu_layer_outputs = tf.nn.relu(inputs + b )
    return (relu_layer_outputs)
    

## Set up a Pooling Layer and Reduce Spatial Size

In [8]:
# creating 2x2 maxpooling layer and reducing spacing size
def create_maxpool2by2_and_reduce_spatial_size(inputs):
    pooling_layers_outputs = tf.nn.max_pool(inputs, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    return (pooling_layers_outputs)
    
    

## Set up fully connected layer and Perform computation: (Input * Weights) + Bias

In [9]:
# Function defines Fully connected layer and computes dot product on input and weights then sums bias

def create_fully_connected_layer_and_compute_dotproduct_plus_bias (inputs, output_size):
    input_size = int(inputs.get_shape()[1])
    W=initialize_weights([input_size, output_size])
    b=initialize_bias([output_size])
    fc_xW_plus_bias_outputs = tf.matmul(inputs, W) + b
    
    return (fc_xW_plus_bias_outputs)

# Phase 1: Build the Convolutional Neural Network

## Create Placeholders for inputs and Labels: x & y_true

In [10]:
# Placeholder for input x
x = tf.placeholder(tf.float32, shape=[None, 784])

In [11]:
# Placeholder for  y_true
y_true = tf.placeholder(tf.float32, [None, 10])

## Reshape the Input  

In [12]:
# Reshaping input x to 28x28
x_image = tf.reshape(x, [-1, 28,28,1])

## Create 1st Convolution Layer, ReLU Layer and Perform Computation: x * W + b

In [13]:
# developing layer one convolution layer and ReLU layer

conv_layer_1_outputs = create_convolution_layer_and_compute_dot_product(x_image, filter_shape = [5,5,1,32])

conv_relu_layer_1_outputs = create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_1_outputs, 
                                                                            filter_shape=[5,5,1,32])
conv_relu_layer_1_outputs.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)])

## Create 1st Pooling Layer and Reduce Spatial Size 

In [14]:
# # developing layer one pooling layer (maxpooling)
# pooling_layer_1_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_1_outputs)

# pooling_layer_1_outputs.shape

## Create 2nd Convolution Layer, ReLU Layer and Perform Computation: x * W + b



In [15]:
# # developing layer two convolution layer and ReLU layer

# conv_layer_2_outputs = create_convolution_layer_and_compute_dot_product(pooling_layer_1_outputs, filter_shape = [5,5,32,64])

# conv_relu_layer_2_outputs = create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_2_outputs, filter_shape=[5,5,32,64])


## Create 2nd Pooling Layer and Reduce Spatial Size 

In [16]:
# # developing layer two pooling layer (maxpooling)
# pooling_layer_2_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_2_outputs)



## Flatten Data Making it Ready to be Fed into 1st Fully Connected Layer


In [17]:
# reshaping the results from pooling layer 2 to 7x7x64 dimensions

conv_relu_layer_1_outputs_flat = tf.reshape(conv_relu_layer_1_outputs, [-1, 7*7*512])

conv_relu_layer_1_outputs_flat.shape

TensorShape([Dimension(None), Dimension(25088)])

## Create Fully Connected Layer ReLU Layer, and Output Data to Dropout Layer

In [18]:
# Attaching Fully connected layer

fc_layer_1_outputs = create_fully_connected_layer_and_compute_dotproduct_plus_bias(conv_relu_layer_1_outputs_flat, 
                                                                                   output_size=1024)
fc_relu_layer_1_outputs = tf.nn.relu(fc_layer_1_outputs)
 



In [19]:
# creating placeholder to hold probabilites for dropout layer and creating dropout function in the network
hold_prob = tf.placeholder(tf.float32)

fc_dropout_outputs = tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=hold_prob)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
# Create Final FC Layer, Compute (x * W + b), and Produce Final Outputs

y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

## Define Loss Function and Calculate Softmax Cross Entropy Loss

In [21]:
# creating softmax activation with logits and cross entropy mean loss function

softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits = y_pred)
cross_entropy_mean = tf.reduce_mean(softmax_cross_entropy_loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## Create an Optimizer to Optimize CNN Model and Set Learning Rate


In [22]:
# Creating an ADAM optimizer

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)



## Create a Trainer to train CNN Model


In [23]:
## building trainer

cnn_trainer = optimizer.minimize(cross_entropy_mean)

## Phase 2: Train and Test CNN Deep Learning Model on MNIST Dataset

###  Create a variable initializer to initilize All Variables 

In [24]:
# Initializing all variables created so that we can use them for modelling

vars_initializer = tf.global_variables_initializer()

### Set the Steps

In [25]:
# Input the number of steps that the model is trained on
steps = 5001

### Run tf.Session() to Train and Test Deep Learning CNN Model

In [26]:
# Using session function we are training and testing the model

with tf.Session() as sess:
    sess.run(vars_initializer)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5 })
        
        if i%100 ==0:
            print ('ON STEP: {}'.format(i))
            print ('ACCURACY: ')

            matches = tf.equal (tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            test_accuracy = sess.run(acc, {x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}  )
            
            print (test_accuracy)
            print('\n')
            
            


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_IN

ON STEP: 0
ACCURACY: 
0.1019


ON STEP: 100
ACCURACY: 
0.9008


ON STEP: 200
ACCURACY: 
0.9165


ON STEP: 300
ACCURACY: 
0.9301


ON STEP: 400
ACCURACY: 
0.9464


ON STEP: 500
ACCURACY: 
0.95


ON STEP: 600
ACCURACY: 
0.9593


ON STEP: 700
ACCURACY: 
0.9556


ON STEP: 800
ACCURACY: 
0.9657


ON STEP: 900
ACCURACY: 
0.9673


ON STEP: 1000
ACCURACY: 
0.9716


ON STEP: 1100
ACCURACY: 
0.9723


ON STEP: 1200
ACCURACY: 
0.9721


ON STEP: 1300
ACCURACY: 
0.9768


ON STEP: 1400
ACCURACY: 
0.9749


ON STEP: 1500
ACCURACY: 
0.9781


ON STEP: 1600
ACCURACY: 
0.9784


ON STEP: 1700
ACCURACY: 
0.9772


ON STEP: 1800
ACCURACY: 
0.9805


ON STEP: 1900
ACCURACY: 
0.9805


ON STEP: 2000
ACCURACY: 
0.9796


ON STEP: 2100
ACCURACY: 
0.9805


ON STEP: 2200
ACCURACY: 
0.979


ON STEP: 2300
ACCURACY: 
0.9807


ON STEP: 2400
ACCURACY: 
0.9829


ON STEP: 2500
ACCURACY: 
0.9822


ON STEP: 2600
ACCURACY: 
0.9813


ON STEP: 2700
ACCURACY: 
0.9832


ON STEP: 2800
ACCURACY: 
0.9818


ON STEP: 2900
ACCURACY: 
0.98